In [83]:
import pandas as pd
import numpy as np
import prophet as pt
import datetime as dt
import os
import matplotlib.pyplot as plt
from prophet.plot import add_changepoints_to_plot


In [84]:
df = pd.read_csv('full_grid.csv')

In [85]:
df.reset_index()
df = df.groupby("grid_num")

In [86]:
def get_hour(datetime_o):
    """
    this function to get the hour of the day to use it as a custom seasonality as an active hours
    """
    return datetime_o.hour
def active_time(t):
    """
    function to determine if the hour is an active hour or not
    """
    if(t in range(4, 16)):
        return 1
    else:
        return 0

In [87]:
def init_prophet():
    #inititating the model
    model = pt.Prophet(changepoint_range=0.9, changepoint_prior_scale=0.001,
                       n_changepoints = 50, weekly_seasonality= 50,
                       daily_seasonality= 50,
                       holidays_prior_scale= 0.1,
                       seasonality_mode = "additive")
    # Adding country holidays
    model.add_country_holidays(country_name= "IT")
    # add monthly seasonality
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

    # Custom seasonality for times in day accoding to active time
    model.add_seasonality(name='daily_active_time', period=1,
                          fourier_order=10, condition_name='active_time')
    return model

In [88]:
def divide_data(iter_num, data, t_period , p_period ):
    """
    function to divide the data set into training and prediction sets
    """
    t_periods = {"week": 144*7, "month": 144*30}
    p_periods = {"day": 144, "week": 144*7}
    
    # For training set
    train_period = t_periods[t_period]
    start_t = p_periods[p_period]*iter_num
    end_t = start_t + train_period
    X_train = data.iloc[start_t:end_t, :]
    
    # For prediction set
    predict_period = p_periods[p_period]
    start_p = end_t
    end_p = start_p + predict_period
    X_predict = data.iloc[start_p:end_p, [0,2]]
    
    full_predict = data.iloc[start_p:end_p, 0:2]
    
    return [X_train, X_predict, full_predict]

In [89]:
def fit_predict(model, train, predict):
    """
    Function to train and predict form a model
    """
    # fitting the model
    model.fit(train)
    #making forecsting for the future 1-day
    forcast = model.predict(predict)
    return forcast

In [91]:
def prediction_dataframe(data, forcast_p, grid_num, iter_num):
    df_p = pd.DataFrame(columns = ["y","yhat","error", "ds", "grid"])
    df_p["y"] = data["y"]
    ### resetting the index to allign the mesurments
    df_p.index = np.arange(df_p.shape[0])
    df_p["yhat"] = forcast_p["yhat"]
    #error for 1week
    df_p["error"] = df_p["y"] - df_p["yhat"]
    df_p["grid"] = grid_num
    forcast_p["ds"].index = np.arange(df_p.shape[0])
    df_p["ds"] = forcast_p["ds"]
    return(df_p)

In [92]:
def nrmse(df_p):
    return np.sqrt((df_p["error"]**2).sum() / df_p.shape[0])/df_p["y"].mean()
def mape(df_p):
    return np.absolute(df_p["error"] / df_p["y"]).sum()/ df_p.shape[0]

In [ ]:
# Get the list of the grids in the city
grids = list(df.groups.keys())
# error dataframe initiation
arrays = [
    ["1 day","1 day", "7 day", "7 days" ] ,
    ["NRMSE", "MAPE", "NRMSE", "MAPE"],
]
# Error data frame columns index
errors = list(zip(*arrays))
#day_errors_dataframe
error_idex_d = pd.MultiIndex.from_tuples(errors[:2], names=["period", "metric"])
error_d = pd.DataFrame(columns = error_idex_d)
#week_errors_dataframe
error_idex_w = pd.MultiIndex.from_tuples(errors[2:], names=["period", "metric"])
error_w = pd.DataFrame(columns = error_idex_w)

# Making a day an a week predictions dataframes to store the values, predictions
# corresponding to each time steps
day_predictions = pd.DataFrame(columns = ["y","yhat","error", "ds", "grid" ])
week_predictions = pd.DataFrame(columns = ["y","yhat","error", "ds", "grid" ])

In [ ]:
for grid in grids[:100]:
    #Get the group of the grid
    df_g = df.get_group(grid)[["time", "rate"]]
    df_g.index = np.arange(df_g.shape[0])
    #converting ds to datetime
    df_g["time"] = pd.to_datetime(df_g["time"])
    #convert the columns o the name of prophet standards
    df_g = df_g.rename(columns = {"time": "ds", "rate": "y"})
    #creatig a new col for the hour
    df_g["hour"] = df_g["ds"].apply(get_hour)
    #col to determine the if the hour is o the active time or not(4am, 4pm)
    df_g["active_time"] = df_g["hour"].apply(active_time)
    #drop the hour colum
    df_g.drop(columns = ["hour"], inplace = True)
    #dividing the dataset
    for i in range(30):
        # Day predictions
        X_train      = divide_data(i, df_g, "month" , "day" )[0]
#         X_train = df_g.iloc[i*144:i*144+144*30, :]
        future_day   = divide_data(i, df_g, "month" , "day" )[1]
#         future_day = df_g.iloc[i*144+144*30:i*144+144*30+144, [0,2]]
        full_predict = divide_data(i, df_g, "month" , "day" )[2]
        period = len(future_day)
        grid_num_d = [grid]*period
        model = init_prophet()
        forcast_day = fit_predict(model, X_train, future_day)
        d_prediction = prediction_dataframe(full_predict,
                                            forcast_day, grid_num_d, i)
        day_predictions = pd.concat([day_predictions, d_prediction])
        nrmse_d = nrmse(d_prediction)
        mape_d = mape(d_prediction)
        error_1d = pd.DataFrame([[nrmse_d,mape_d]],columns = error_idex_d)
        error_d = pd.concat([error_d, error_1d])
        print("========== forcasting for {} day for grid {} is done =========".format(i+1, grid))

        
    for w in range(4):
        # week predictions
        X_train      = divide_data(w, df_g, "month" , "week" )[0]
        future_week  = divide_data(w, df_g, "month" , "week" )[1]
        full_predict = divide_data(w, df_g, "month" , "week" )[2]
        period = len(future_week)
        grid_num_w = [grid]*period
        model = init_prophet()
        forcast_week = fit_predict(model, X_train, future_week)
        w_prediction = prediction_dataframe(full_predict,
                                            forcast_week, grid_num_w, w)
        week_predictions = pd.concat([week_predictions, w_prediction])
        nrmse_w = nrmse(w_prediction)
        mape_w = mape(w_prediction)
        error_1w = pd.DataFrame([[nrmse_w,mape_w]],columns = error_idex_w)
        error_w = pd.concat([error_w, error_1w])
        print("========== forcasting for {} week for grid {} is done".format(w+1, grid))

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


========== forcasting for 1 day for grid 38 is done =========


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


========== forcasting for 2 day for grid 38 is done =========


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


...

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


========== forcasting for 12 day for grid 156 is done =========


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


========== forcasting for 13 day for grid 156 is done =========


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


,y,yhat,error,ds,grid
0,14.522020,14.880459,-0.358439,2013-12-01 00:00:00,38
1,16.606790,14.224551,2.382239,2013-12-01 00:10:00,38
2,14.151956,13.709881,0.442074,2013-12-01 00:20:00,38
3,13.085586,12.989645,0.095942,2013-12-01 00:30:00,38
4,11.664906,12.705049,-1.040143,2013-12-01 00:40:00,38
...,...,...,...,...,...
1003,19.969331,22.402530,-2.433198,2013-12-28 23:10:00,38
1004,19.277136,21.593983,-2.316847,2013-12-28 23:20:00,38
1005,14.762119,20.331749,-5.569630,2013-12-28 23:30:00,38
1006,20.400707,19.415173,0.985533,2013-12-28 23:40:00,38


In [82]:
error_d

period     1 day          
metric     NRMSE      MAPE
0       0.309171  0.245530
0       0.309171  0.245530
0       0.349472  0.376993
0       0.197721  0.103962
0       0.176732  0.125109
..           ...       ...
0       0.466087  0.271060
0       0.237821  0.309882
0       0.356065  0.207745
0       0.463833  0.455734
0       0.297660  0.173793

[121 rows x 2 columns]

In [56]:
error_d

period     1 day          
metric     NRMSE      MAPE
0       0.309171  0.245530
0       0.309171  0.245530
0       0.349472  0.376993
0       0.197721  0.103962
0       0.176732  0.125109
..           ...       ...
0       0.466087  0.271060
0       0.237821  0.309882
0       0.356065  0.207745
0       0.463833  0.455734
0       0.297660  0.173793

[91 rows x 2 columns]